In [3]:
import csv

# Define the path to your CSV file
csv_file_path = 'Dataset/production.csv'

# Define the column you want to analyze (change this to the desired column)
column_to_analyze = 'Module'  # Replace 'ColumnName' with the actual column name

# Initialize variables to store the statistics
total_values = 0
sum_of_values = 0
min_value = None
max_value = None

# Open the CSV file and read its contents
with open(csv_file_path, 'r') as csv_file:
    csv_reader = csv.DictReader(csv_file)

    # Iterate through each row in the CSV file
    for row in csv_reader:
        # Check if the column value exists in the row
        if column_to_analyze in row:
            # Assuming the column contains numeric values
            try:
                value = float(row[column_to_analyze])
                total_values += 1
                sum_of_values += value
                if min_value is None or value < min_value:
                    min_value = value
                if max_value is None or value > max_value:
                    max_value = value
            except ValueError:
                print(f"Skipping row with non-numeric value in '{column_to_analyze}': {row[column_to_analyze]}")

# Calculate the average
average_value = sum_of_values / total_values if total_values > 0 else 0

# Print the analysis results
print(f"Total values in '{column_to_analyze}': {total_values}")
print(f"Sum of values in '{column_to_analyze}': {sum_of_values}")
print(f"Average value in '{column_to_analyze}': {average_value}")
print(f"Minimum value in '{column_to_analyze}': {min_value}")
print(f"Maximum value in '{column_to_analyze}': {max_value}")


Skipping row with non-numeric value in 'Module': AL01
Skipping row with non-numeric value in 'Module': 
Skipping row with non-numeric value in 'Module': AL01
Skipping row with non-numeric value in 'Module': 
Skipping row with non-numeric value in 'Module': AL02
Skipping row with non-numeric value in 'Module': AL02
Skipping row with non-numeric value in 'Module': AL02
Skipping row with non-numeric value in 'Module': AL02
Skipping row with non-numeric value in 'Module': AL02
Skipping row with non-numeric value in 'Module': AL02
Skipping row with non-numeric value in 'Module': AL02
Skipping row with non-numeric value in 'Module': AL02
Skipping row with non-numeric value in 'Module': AL02
Skipping row with non-numeric value in 'Module': AL02
Skipping row with non-numeric value in 'Module': AL02
Skipping row with non-numeric value in 'Module': AL02
Skipping row with non-numeric value in 'Module': AL02
Skipping row with non-numeric value in 'Module': AL02
Skipping row with non-numeric value 